In [1]:
import sqlparse

from time import perf_counter

from django.test import RequestFactory
from django.db import connection, reset_queries

from cast.models.repository import BlogIndexRepository

In [2]:
def count_queries(func):
    reset_queries()
    renditions = list(func())
    return len(connection.queries)


def show_queries(queries):
    queries = sorted(queries, key=lambda x: x["time"], reverse=True)
    for num, query in enumerate(queries, 1):
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        duration = query["time"]
        print(f"{num}. query | duration: {duration}")
        print(formatted_sql)
        print("---------------")
        

def blocker(*_args):
    """Get a traceback when a query is executed."""
    raise Exception("No database access allowed here.")

In [ ]:
# blog = Blog.objects.get(slug="ephes_blog")
blog = Blog.objects.get(slug="das_claas_und_nora_blog")

In [4]:
rf = RequestFactory()
request = rf.get(blog.get_url())
request.htmx = False

In [5]:
%%time
reset_queries()
cachable_data = BlogIndexRepository.data_for_blog_index_cachable(request=request, blog=blog)

CPU times: user 55.2 ms, sys: 3.34 ms, total: 58.5 ms
Wall time: 71.7 ms


In [10]:
%%time
repository = BlogIndexRepository.create_from_cachable_data(data=cachable_data)

CPU times: user 28.6 ms, sys: 1.82 ms, total: 30.5 ms
Wall time: 29 ms


In [11]:
len(connection.queries)

18

In [12]:
show_queries(connection.queries)

1. query | duration: 0.002
SELECT "cast_postcategory"."slug",
       "cast_postcategory"."name",
       COUNT("cast_post_categories"."post_id") FILTER (
                                                       WHERE "cast_post_categories"."post_id" IN
                                                           (SELECT U0."page_ptr_id"
                                                            FROM "cast_post" U0
                                                            INNER JOIN "wagtailcore_page" U1 ON (U0."page_ptr_id" = U1."id")
                                                            WHERE (U1."live"
                                                                   AND U1."path"::text LIKE '000100010001%'
                                                                   AND U1."depth" >= 3
                                                                   AND NOT (U0."page_ptr_id" = 3)))) AS "num_posts"
FROM "cast_postcategory"
LEFT OUTER JOIN "cast_post_categories" ON ("cast

In [14]:
%%time
reset_queries()
# with connection.execute_wrapper(blocker):
response = blog.serve(request, repository=repository).render()
print(len(connection.queries))

0
CPU times: user 129 ms, sys: 32.6 ms, total: 162 ms
Wall time: 167 ms


In [15]:
response.context_data

{'page': <Blog: Claas und Nora sagen Hallo>,
 'self': <Blog: Claas und Nora sagen Hallo>,
 'request': <WSGIRequest: GET '/blogs/das_claas_und_nora_blog/'>,
 'repository': <cast.models.repository.BlogIndexRepository at 0x1142ed910>,
 'ellipsis': '…',
 'page_number': 1,
 'page_range': [1, 2, 3, '…', 64],
 'is_paginated': True,
 'has_previous': False,
 'previous_page_number': None,
 'has_next': True,
 'next_page_number': 2,
 'object_list': [<Post: April 2024-04-08>,
  <Post: April 2024-04-01>,
  <Post: März 2024-03-25>,
  <Post: März 2024-03-18>,
  <Post: März 2024-03-11>],
 'filterset': <cast.filters.PostFilterset at 0x1142ecd40>,
 'template_base_dir': 'bootstrap5',
 'use_audio_player': False,
 'root_nav_links': [('/blogs/das_claas_und_nora_blog/',
   'Claas und Nora sagen Hallo'),
  ('/blogs/ephes_blog/', 'Ephes Blog')],
 'posts': [<Post: April 2024-04-08>,
  <Post: April 2024-04-01>,
  <Post: März 2024-03-25>,
  <Post: März 2024-03-18>,
  <Post: März 2024-03-11>],
 'blog': <Blog: Claas

# Profile Response Rendering

In [19]:
import cProfile
import pstats

In [17]:
%%time
profiler = cProfile.Profile()
profiler.enable()
response = blog.serve(request, repository=repository).render()
profiler.disable()

CPU times: user 107 ms, sys: 21.8 ms, total: 128 ms
Wall time: 127 ms


In [20]:
stats = pstats.Stats(profiler)
# stats.sort_stats('cumulative').print_stats()